Note that need tensorflow 1.15.2

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import scipy.io
#from scipy.interpolate import griddata
import time
#from itertools import product, combinations

In [2]:
np.random.seed(1234)
tf.set_random_seed(1234)

In [35]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, t, u1, u2,u3,u4,u5,u6, F, tau, layers):
        
        X = t
        
        self.lb = X.min(0)
        self.ub = X.max(0)
                
        self.X = X
        
        self.t = X[:,0:1]
        self.F = F[:,0:1]
        self.tau = tau[:,0:1]
        
        self.u1 = u1
        self.u2 = u2
        self.u3 = u3
        self.u4 = u4
        self.u5 = u5
        self.u6 = u6
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)        
        
        # Initialize parameters
        self.bv = tf.Variable([4.0], dtype=tf.float32)    
        self.bo = tf.Variable([10.0], dtype=tf.float32)    
        #self.m = tf.Variable([11.0], dtype=tf.float32)  
        #self.I = tf.Variable([10.0], dtype=tf.float32)    
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.F_tf = tf.placeholder(tf.float32, shape=[None, self.F.shape[1]])
        self.tau_tf = tf.placeholder(tf.float32, shape=[None, self.tau.shape[1]])
        
        self.u1_tf = tf.placeholder(tf.float32, shape=[None, self.u1.shape[1]])
        self.u2_tf = tf.placeholder(tf.float32, shape=[None, self.u2.shape[1]])
        self.u3_tf = tf.placeholder(tf.float32, shape=[None, self.u3.shape[1]])
        self.u4_tf = tf.placeholder(tf.float32, shape=[None, self.u4.shape[1]])
        self.u5_tf = tf.placeholder(tf.float32, shape=[None, self.u5.shape[1]])
        self.u6_tf = tf.placeholder(tf.float32, shape=[None, self.u6.shape[1]])
        
        self.u1_pred,self.u2_pred,self.u3_pred,self.u4_pred,self.u5_pred,self.u6_pred,\
        self.f_u1_pred ,self.f_u2_pred,self.f_u3_pred,self.f_u4_pred,self.f_u5_pred,self.f_u6_pred = \
            self.net_NS(self.t_tf, self.F_tf, self.tau_tf)
        
        self.loss = tf.reduce_sum(tf.square(self.u1_tf - self.u1_pred)) + \
                    tf.reduce_sum(tf.square(self.u2_tf - self.u2_pred)) + \
                    tf.reduce_sum(tf.square(self.u3_tf - self.u3_pred)) + \
                    tf.reduce_sum(tf.square(self.u4_tf - self.u4_pred)) + \
                    tf.reduce_sum(tf.square(self.u5_tf - self.u5_pred)) + \
                    tf.reduce_sum(tf.square(self.u6_tf - self.u6_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u1_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u2_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u3_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u4_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u5_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u6_pred))
                    
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})        
        
        self.optimizer_Adam = tf.train.AdamOptimizer(learning_rate=0.01)
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = X
        #H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
        
    def net_NS(self, t, F, tau):
        bv = self.bv
        bo = self.bo
        m = 11.0
        I = 10.0
        g = 9.8
        
        state = self.neural_net(tf.concat([t], 1), self.weights, self.biases)
        u1 = state[:,0:1]
        u2 = state[:,1:2]
        u3 = state[:,2:3]
        u4 = state[:,3:4]
        u5 = state[:,4:5]
        u6 = state[:,5:6]
        
        
        u1_t = tf.gradients(u1, t)[0]
        u2_t = tf.gradients(u2, t)[0]
        u3_t = tf.gradients(u3, t)[0]
        u4_t = tf.gradients(u4, t)[0]
        u5_t = tf.gradients(u5, t)[0]
        u6_t = tf.gradients(u6, t)[0]

        f_u1 = u1_t - u1 
        f_u2 = u2_t - u2
        f_u3 = u3_t - u3
        f_u4 = u4_t + u4*bv/m + tf.math.multiply(tf.math.sin(u3)/m, tf.convert_to_tensor(F))
        f_u5 = u5_t + u5*bv/m + g - tf.math.multiply(tf.math.cos(u3)/m, tf.convert_to_tensor(F))
        f_u6 = u6_t + u6*bo/I - tf.convert_to_tensor(tau)/I
        
        return u1, u2, u3, u4, u5, u6, f_u1, f_u2, f_u3, f_u4, f_u5, f_u6
    
    def callback(self, loss, bv, bo):
        print('Loss: %.3e, bv: %.2f, bo: %.2f' % (loss, bv, bo))
      
    def train(self, nIter): 

        tf_dict = {self.t_tf: self.t, self.u1_tf: self.u1, self.u2_tf: self.u2, self.u3_tf: self.u3, 
                   self.u4_tf: self.u4, self.u5_tf: self.u5, self.u6_tf: self.u6, self.F_tf: self.F, self.tau_tf: self.tau}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 100 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                bv_value = self.sess.run(self.bv)
                bo_value = self.sess.run(self.bo)
                print('It: %d, Loss: %.3e, bv: %.2f, bo: %.2f, Time: %.2f' % 
                      (it, loss_value, bv_value, bo_value, elapsed))
                start_time = time.time()
            
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss, self.bv, self.bo],
                                loss_callback = self.callback)
            
    
    #def predict(self, t_star):
        
    #    tf_dict = {self.x_tf: x_star, self.y_tf: y_star, self.t_tf: t_star}
        
    #    u_star = self.sess.run(self.u_pred, tf_dict)
    #    v_star = self.sess.run(self.v_pred, tf_dict)
    #    p_star = self.sess.run(self.p_pred, tf_dict)
        
    #    return u_star, v_star, p_star
              

In [36]:
# Load Data

# read data

filepath = '../Data/'
DATA_dict = dict()
num_traj = 40
for i in range(num_traj):
    num = i+41;
    if num >= 10:
        str_num = str(num) # no leading zero
    else:
        str_num = '0'+str(num) # add leading zero
    filename = 'trajdata_'+str_num+'.csv'
    with open(filepath+filename, 'r') as f:
        # remove top row, and store the rest in dictionary
        DATA_dict[i] = np.genfromtxt(f, dtype='f4', delimiter=',', skip_header=1)

In [37]:
idx = 23
t = DATA_dict[idx][:,0:1]
u1 = DATA_dict[idx][:,1:2]
u2 = DATA_dict[idx][:,2:3]
u3 = DATA_dict[idx][:,3:4]
u4 = DATA_dict[idx][:,4:5]
u5 = DATA_dict[idx][:,5:6]
u6 = DATA_dict[idx][:,6:7]
F = DATA_dict[idx][:,7:8]
tau = DATA_dict[idx][:,8:9]

In [38]:
layers = [1, 10,10,10, 6]

# Training
model = PhysicsInformedNN(t, u1, u2, u3, u4, u5, u6, F, tau, layers)
model.train(200000)
    

Device mapping:

It: 0, Loss: 4.507e+05, bv: 4.00, bo: 10.00, Time: 0.44
It: 100, Loss: 4.134e+05, bv: 3.95, bo: 9.98, Time: 0.87
It: 200, Loss: 3.922e+05, bv: 3.87, bo: 10.15, Time: 0.90
It: 300, Loss: 3.731e+05, bv: 3.82, bo: 10.34, Time: 0.90
It: 400, Loss: 3.536e+05, bv: 3.84, bo: 10.53, Time: 0.89
It: 500, Loss: 3.378e+05, bv: 3.93, bo: 10.73, Time: 0.88
It: 600, Loss: 3.309e+05, bv: 4.07, bo: 10.92, Time: 0.87
It: 700, Loss: 3.315e+05, bv: 4.23, bo: 11.11, Time: 0.89
It: 800, Loss: 3.301e+05, bv: 4.40, bo: 11.27, Time: 0.87
It: 900, Loss: 3.251e+05, bv: 4.57, bo: 11.41, Time: 0.88
It: 1000, Loss: 3.199e+05, bv: 4.74, bo: 11.54, Time: 0.87
It: 1100, Loss: 3.147e+05, bv: 4.90, bo: 11.66, Time: 0.88
It: 1200, Loss: 2.933e+05, bv: 5.05, bo: 11.75, Time: 0.87
It: 1300, Loss: 2.861e+05, bv: 5.17, bo: 11.78, Time: 0.87
It: 1400, Loss: 2.757e+05, bv: 5.27, bo: 11.79, Time: 0.87
It: 1500, Loss: 2.661e+05, bv: 5.34, bo: 11.80, Time: 0.89
It: 1600, Loss: 2.597e+05, bv: 5.38, bo: 11.80, Time

KeyboardInterrupt: 

In [39]:
print('bv ',model.sess.run(model.bv)[0])
print('bo ',model.sess.run(model.bo)[0])

bv  4.929752
bo  11.0624695
m  10.217946
I  11.138924


In [ ]:
N_train = 5000
    
layers = [1, 10, 10, 6]

# Load Data
data = scipy.io.loadmat('../Data/cylinder_nektar_wake.mat')
           
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2
    
N = X_star.shape[0]
T = t_star.shape[0]
    
# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T
    
UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T
    
x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1
    
u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1
    
    ######################################################################
    ######################## Noiseles Data ###############################
    ######################################################################
# Training Data    
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

# Training
model = PhysicsInformedNN(x_train, y_train, t_train, u_train, v_train, layers)
model.train(200000)
    
# Test Data
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]
    
u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]
    
# Prediction
u_pred, v_pred, p_pred = model.predict(x_star, y_star, t_star)
lambda_1_value = model.sess.run(model.lambda_1)
lambda_2_value = model.sess.run(model.lambda_2)
    
# Error
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100
    
print('Error u: %e' % (error_u))    
print('Error v: %e' % (error_v))    
print('Error p: %e' % (error_p))    
print('Error l1: %.5f%%' % (error_lambda_1))                             
print('Error l2: %.5f%%' % (error_lambda_2))                  